In [2]:
import pandas as pd
from pandas import Series, DataFrame
from os.path import join, normpath
from os import path
from collections import namedtuple
import numpy as np
import datetime as dt

import roamability as rb

sponsor_object = namedtuple('Sponsor', ['name', 'mccmnc', 'sponsor_id'])

#######################################################################################
# Define variables here
#######################################################################################

rep_date_start = dt.date(2020,2,1) # Including
rep_date_end = dt.date(2020,2,29) # Including

downloads = 'C:/Users/balob/Downloads'

s1_sponsor = sponsor_object('S1', 42501, 1)
s2_sponsor = sponsor_object('S2', 26006, 2)
s4_sponsor = sponsor_object('S4', 45403, 4)
s5_sponsor = sponsor_object('S5', 51503, 5)

use_value = 'Revenue' # 'Cost', 'Revenue', 'Profit'

sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor]

#######################################################################################

#######################################################################################

In [3]:
# Get data from OCS DB и Подготовка данных

sql_srt = """
SELECT
CONVERT(VARCHAR(10), u.USAGE_DATE, 103) AS Day
,s.DISPLAY_NAME AS Sponsor
,mm.NETWORK_ID AS VisitedNetworkId
,r.RESELLER_ID AS Customer
,SUM(uc.COST) AS Cost
,SUM(uc.CHARGE) AS Revenue
,ut.USAGE_TYPE AS UsageType
,MIN(uc.BALANCE) AS BALANCE
FROM [10028_usage].dbo.usage u
INNER JOIN [10028_usage].dbo.USAGE_CHARGES uc ON u.USAGE_ID = uc.USAGE_ID
INNER JOIN [10028].dbo.USAGE_TYPE ut ON u.USAGE_TYPE = ut.USAGE_TYPE_ID
INNER JOIN [10028].dbo.RESELLERS r ON uc.ACCOUNT_ID = r.ACCOUNT_ID
INNER JOIN [10028].dbo.RDB_NETWORK_IMSI_PREFIXES mm ON u.MCC = mm.MCC AND u.MNC = mm.MNC
INNER JOIN [10028].dbo.SPONSORS s ON u.GUIDING_INFO LIKE CONCAT(s.IMSI_PREFIX, '%')
WHERE
uc.CHARGE_ENTITY_TYPE = 'Reseller'
AND u.usage_date BETWEEN '{start_date}' AND '{end_date}'
GROUP BY
CONVERT(VARCHAR(10), u.USAGE_DATE, 103)
,ut.USAGE_TYPE
,r.RESELLER_ID
,r.RESELLER_NAME
,mm.NETWORK_ID
,s.DISPLAY_NAME""".format(start_date = rep_date_start.strftime('%Y-%m-%d %H:%M:%S'),
                          end_date = (rep_date_end + pd.Timedelta('1D')).strftime('%Y-%m-%d %H:%M:%S'))

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_ocs_source = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    
df_ocs_source.replace({'Data':'DATA','Incoming Text':'MTSMS','Outgoing Call':'MOC','Outgoing Text':'MOSMS'}, inplace=True)
    
# Подготовка данных

df_ocs_processed = df_ocs_source.copy()

# Подготовка колонки Profit
df_ocs_processed['Profit'] = df_ocs_processed['Revenue'] - df_ocs_processed['Cost']

# Подготовка широкого DF с заданием используемого значения (Cost, Revenue, Profit)
df_ocs_processed = df_ocs_processed.pivot_table(values=use_value,
                                                index=['Day','Sponsor','VisitedNetworkId','Customer'],
                                                columns='UsageType', aggfunc=np.sum).reset_index()

# Обработка значения даты с добавлением дополнительных полей даты
df_ocs_processed.Day = pd.to_datetime(df_ocs_processed.Day, format='%d/%m/%Y')
df_ocs_processed['Month'] = df_ocs_processed.Day.dt.strftime('%Y-%m')

# Добавление поля TOTAL и выборка используемых в дальнейшем полей
df_ocs_processed['TOTAL'] = df_ocs_processed[['MOC','MOSMS','MTSMS','DATA']].sum(axis=1)
df_ocs_processed['MTC'] = 0
usage_type_names = ['MOC','MTC','MOSMS','MTSMS','DATA', 'TOTAL']
df_ocs_processed[usage_type_names] = df_ocs_processed[usage_type_names].astype('float64')
df_ocs_processed = df_ocs_processed[
    ['Day','Month','Sponsor','VisitedNetworkId','Customer','MOC','MTC','MOSMS','MTSMS','DATA', 'TOTAL']].fillna(0)

df_ocs_processed.head(3)

UsageType,Day,Month,Sponsor,VisitedNetworkId,Customer,MOC,MTC,MOSMS,MTSMS,DATA,TOTAL
0,2020-02-01,2020-02,S1,12,172844,0.028,0.0,0.0,0.0,0.00000,0.02800
1,2020-02-01,2020-02,S1,12,172868,0.000,0.0,0.0,0.0,0.10048,0.10048
2,2020-02-01,2020-02,S1,27,172718,0.000,0.0,0.0,0.0,5.05897,5.05897


In [4]:
# Получение данных из BSS и Подготовка данных

sql_srt="""
SELECT * FROM
(
SELECT
Day,SponsorPrefix AS Sponsor,VisitedNetworkId,Customer,Cost,Revenue
,'DATA' AS UsageType
FROM aggreg.GPRS_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
UNION ALL
SELECT
Day,SponsorPrefix AS Sponsor,VisitedNetworkId,Customer,Cost,Revenue
,CASE WHEN Sms = 0 THEN 'MOC' WHEN Sms = 1 THEN 'MOSMS' END AS UsageType
FROM aggreg.MOC_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
UNION ALL
SELECT
Day,SponsorPrefix AS Sponsor,VisitedNetworkId,Customer,Cost,Revenue
,CASE WHEN Sms = 0 THEN 'MTC' WHEN Sms = 1 THEN 'MTSMS' END AS UsageType
FROM aggreg.MTC_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
) t
""".format(start_date = rep_date_start.strftime('%Y%m%d'), end_date = rep_date_end.strftime('%Y%m%d'))

with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df_bss_source = pd.read_sql_query(sql_srt, cnxn)
# df_bss_source.to_csv(join(downloads, output_bss_usage_file), index=False)

# Подготовка данных и объединение со справочниками

df_bss_processed = df_bss_source.copy()

# Подготовка колонки Profit
df_bss_processed['Profit'] = df_bss_processed['Revenue'] - df_bss_processed['Cost']

# Замена префикса Спонсора на символьное обозначение? S1, S2...
sponsor_dict = dict((sponsor.mccmnc, sponsor.name) for sponsor in sponsors)
df_bss_processed.Sponsor.replace(sponsor_dict, inplace=True)

# Подготовка широкого DF с заданием используемого значения (Cost, Revenue, Profit)
df_bss_processed = df_bss_processed.pivot_table(values=use_value,
                                                index=['Day','Sponsor','VisitedNetworkId','Customer'],
                                                columns='UsageType', aggfunc=np.sum).reset_index()

# Обработка значения даты с добавлением дополнительных полей даты
df_bss_processed.Day = pd.to_datetime(df_bss_processed.Day, format='%Y%m%d')
df_bss_processed['Month'] = df_bss_processed.Day.dt.strftime('%Y-%m')

# Добавление поля TOTAL и выборка используемых в дальнейшем полей
df_bss_processed['TOTAL'] = df_bss_processed[['MOC','MTC','MOSMS','MTSMS','DATA']].sum(axis=1)
df_bss_processed = df_bss_processed[
    ['Day','Month','Sponsor','VisitedNetworkId','Customer','MOC','MTC','MOSMS','MTSMS','DATA', 'TOTAL']].fillna(0)

df_bss_processed.head(3)

UsageType,Day,Month,Sponsor,VisitedNetworkId,Customer,MOC,MTC,MOSMS,MTSMS,DATA,TOTAL
0,2020-02-01,2020-02,S1,12,172716,0.067,0.0,0.0,0.0,1.497290,1.564290
1,2020-02-01,2020-02,S1,12,172717,0.000,0.0,0.0,0.0,0.191163,0.191163
2,2020-02-01,2020-02,S1,12,172844,0.025,0.0,0.0,0.0,0.000000,0.025000


In [5]:
# Загрузка справочников из OCS

# sql_srt='SELECT network_id AS VisitedNetworkId, MAX(tadig_code) AS Tadig FROM RDB_TADIG_CODES GROUP BY network_id;'

sql_srt="""SELECT t.network_id AS VisitedNetworkId, MAX(c.country_name) AS Country, MAX(t.tadig_code) AS Tadig
FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
GROUP BY t.network_id;"""

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
display(df_tadig.head(3))

sql_srt='SELECT reseller_id AS Customer, reseller_name AS CustomerName FROM RESELLERS;'
with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_customer = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    
df_customer.replace({'STI - Telinta':'STI'}, inplace=True)

display(df_customer.head(3))

,VisitedNetworkId,Country,Tadig
0,4,Afghanistan,AFGAW
1,5,Afghanistan,AFGAR
2,6,Afghanistan,AFGEA


,Customer,CustomerName
0,172711,Mondicon
1,172716,Cloud 9 TAP
2,172717,KnowRoaming IMSI-RANGE TAP


In [6]:
# Агрегация по времени

aggregation_time = ['Month'] # Month, Day
aggregation_list = ['Customer'] # 'Sponsor','VisitedNetworkId','Customer'
usage_type = ['TOTAL']

# Сравнение

df_temp = pd.merge(
    df_bss_processed.groupby(aggregation_list + aggregation_time, as_index=False)[usage_type].sum(),
    df_ocs_processed.groupby(aggregation_list + aggregation_time, as_index=False)[usage_type].sum(),
    how='outer', on = aggregation_list + aggregation_time, suffixes=['_BSS','_OCS']
).dropna()

cols = (aggregation_time + ['CustomerName'] + aggregation_list + ['TOTAL_BSS', 'TOTAL_OCS'])
cols.remove('Customer')
pd.merge(df_temp, df_customer, how='left', on='Customer')[cols]

,Month,CustomerName,TOTAL_BSS,TOTAL_OCS
0,2020-02,Porto Seguro DMI RT,199.574098,222.61725
1,2020-02,Netmore DMI RT,1385.015199,1451.71694
2,2020-02,Roamability Test RT,216.327802,242.67453
3,2020-02,uCloudLink RT,0.000175,0.00023
4,2020-02,Mondicon SIM RT,972.401705,1038.42258
5,2020-02,Clay RT,472.420911,803.68163
6,2020-02,RedTea RT,2823.975161,3185.91455
7,2020-02,Nextel DMI RT,129.377559,160.96347
8,2020-02,Maxcom RT,170.474698,165.29911
9,2020-02,Dialoq - Genesisclub RT,9545.354825,14308.17113


In [54]:
# Детализация полная

aggregation_time = ['Month'] # Month, Day
aggregation_list = ['Customer','Sponsor','VisitedNetworkId']
usage_type = ['MOC','MTC','MOSMS','MTSMS','DATA']
customer = ['Dialoq - Genesisclub RT']

output_diff_file = f"bss_ocs_diff_full_{rep_date_start.strftime('%Y%m%d')}_{rep_date_end.strftime('%Y%m%d')}.xlsx"

# Сравнение

df_temp = pd.merge(
    df_bss_processed.groupby(aggregation_list + aggregation_time, as_index=False)[usage_type].sum(),
    df_ocs_processed.groupby(aggregation_list + aggregation_time, as_index=False)[usage_type].sum(),
    how='outer', on = aggregation_list + aggregation_time, suffixes=['_BSS','_OCS']
).fillna(0)

df_temp = pd.merge(df_temp, df_customer, how='left', on='Customer')
df_temp = pd.merge(df_temp, df_tadig, how='left', on='VisitedNetworkId')
df_temp.drop(['VisitedNetworkId','Customer'], axis=1, inplace=True)

df_temp = df_temp.loc[df_temp.CustomerName.isin(customer)]

cols = aggregation_time + ['Country','Sponsor'] + ['Tadig','CustomerName',
                                              'MOC_BSS','MOC_OCS','MTC_BSS','MTC_OCS','MOSMS_BSS','MOSMS_OCS',
                                              'MTSMS_BSS','MTSMS_OCS','DATA_BSS','DATA_OCS']

display(df_temp[cols].head())
df_temp[cols].to_excel(join(downloads, output_diff_file), index=False)

,Month,Country,Sponsor,Tadig,CustomerName,MOC_BSS,MOC_OCS,MTC_BSS,MTC_OCS,MOSMS_BSS,MOSMS_OCS,MTSMS_BSS,MTSMS_OCS,DATA_BSS,DATA_OCS
834,2020-02,Albania,S1,ALBVF,Dialoq - Genesisclub RT,0.00000,0.00000,0.0,0.0,0.1320,0.0360,0.0,0.0,10.431427,10.47404
835,2020-02,Armenia,S1,ARM01,Dialoq - Genesisclub RT,0.00000,0.00000,0.0,0.0,2.3000,1.5000,0.0,0.0,8.664562,9.12208
836,2020-02,Australia,S1,AUSOP,Dialoq - Genesisclub RT,0.00000,0.00000,0.0,0.0,0.2160,0.1800,0.0,0.0,15.806335,17.84841
837,2020-02,Austria,S1,AUTPT,Dialoq - Genesisclub RT,0.00000,0.00000,0.0,0.0,8.6100,8.7300,0.0,0.0,228.078019,241.53234
838,2020-02,Austria,S1,AUTHU,Dialoq - Genesisclub RT,0.19406,0.19406,0.0,0.0,1.6244,1.6864,0.0,0.0,59.649170,67.20925


In [57]:
# Детализация с подкраской

aggregation_time = ['Month'] # Month, Day
aggregation_list = ['Customer','Sponsor','VisitedNetworkId'] # 'Sponsor','VisitedNetworkId','Customer'
usage_type = ['MOC','MTC','MOSMS','MTSMS','DATA']
customer = ['Dialoq - Genesisclub RT']
rel_tolerance = 0.5 # 1 - 100% от Chgarge Example: Cost = 10 USD, Charge < 10 USD

output_diff_file = f"bss_ocs_diff_colored_{rep_date_start.strftime('%Y%m%d')}_{rep_date_end.strftime('%Y%m%d')}.xlsx"

# aggregation_list.extend(aggregation_time)

# Сравнение

df_temp = pd.merge(
    df_bss_processed.groupby(aggregation_list + aggregation_time, as_index=False)[usage_type].sum(),
    df_ocs_processed.groupby(aggregation_list + aggregation_time, as_index=False)[usage_type].sum(),
    how='outer', on = aggregation_list + aggregation_time, suffixes=['_BSS','_OCS']
).fillna(0)

df_temp = pd.merge(df_temp, df_customer, how='left', on='Customer')

df_temp = df_temp.loc[df_temp.CustomerName.isin(customer)]

cols = aggregation_time + ['Country'] + aggregation_list + ['Tadig','CustomerName',
                                              'MOC_BSS','MOC_OCS','MTC_BSS','MTC_OCS','MOSMS_BSS','MOSMS_OCS',
                                              'MTSMS_BSS','MTSMS_OCS','DATA_BSS','DATA_OCS']

if 'Customer' in cols:
    cols.remove('Customer')
if 'VisitedNetworkId' in cols:
    cols.remove('VisitedNetworkId')

df_temp = pd.merge(df_temp, df_tadig, how='left', on='VisitedNetworkId')[cols]

df_diff = df_temp[
    ~np.isclose(df_temp.MOC_BSS, df_temp.MOC_BSS, rtol=rel_tolerance, equal_nan=True) |\
    ~np.isclose(df_temp.MTC_BSS, df_temp.MTC_BSS, rtol=rel_tolerance, equal_nan=True) |\
    ~np.isclose(df_temp.MOSMS_BSS, df_temp.MOSMS_OCS, rtol=rel_tolerance, equal_nan=True) |\
    ~np.isclose(df_temp.MTSMS_BSS, df_temp.MTSMS_OCS, rtol=rel_tolerance, equal_nan=True) |\
    ~np.isclose(df_temp.DATA_BSS, df_temp.DATA_OCS, rtol=rel_tolerance, equal_nan=True)
]

def colorize(s):
    color_list = []
    for tariff_type in [['MOC_BSS','MOC_BSS'], ['MTC_BSS','MTC_BSS'],
                        ['MOSMS_BSS','MOSMS_OCS'], ['MTSMS_BSS','MTSMS_OCS'], ['DATA_BSS','DATA_OCS']]:
        if np.isclose(s[tariff_type[0]], s[tariff_type[1]], rtol=rel_tolerance, equal_nan=True):
            color_list.extend(['', ''])
        elif s[tariff_type[0]] > s[tariff_type[1]]:
            color_list.extend(['background-color: yellow', 'background-color: red'])
        else:
            color_list.extend(['background-color: yellow', 'background-color: #81EF21'])
    return color_list

df_diff_style = df_diff.style.apply(colorize,
                            subset=['MOC_BSS','MOC_OCS','MTC_BSS','MTC_OCS','MOSMS_BSS','MOSMS_OCS',
                          'MTSMS_BSS','MTSMS_OCS','DATA_BSS','DATA_OCS'], axis=1).hide_index()

display(df_diff_style)
df_diff_style.to_excel(join(downloads, output_diff_file), index=False)

Month,Country,Sponsor,Tadig,CustomerName,MOC_BSS,MOC_OCS,MTC_BSS,MTC_OCS,MOSMS_BSS,MOSMS_OCS,MTSMS_BSS,MTSMS_OCS,DATA_BSS,DATA_OCS
2020-02,Albania,S1,ALBVF,Dialoq - Genesisclub RT,0,0,0,0,0.132,0.036,0,0,10.4314,10.474
2020-02,Armenia,S1,ARM01,Dialoq - Genesisclub RT,0,0,0,0,2.3,1.5,0,0,8.66456,9.12208
2020-02,Barbados,S1,BRBCW,Dialoq - Genesisclub RT,0,0,0,0,0,0,0,0,0.744729,2.32072
2020-02,Belarus,S1,BLRMD,Dialoq - Genesisclub RT,2.196,2.196,0,0,2.475,0.8,0,0,73.5437,81.0027
2020-02,Chile,S1,CHLTM,Dialoq - Genesisclub RT,0,0,0,0,0,0,0,0,3.13648,13.2642
2020-02,Colombia,S1,COLTM,Dialoq - Genesisclub RT,0,0,0,0,0.12,0,0,0,25.8452,29.3597
2020-02,Ghana,S1,GHAGT,Dialoq - Genesisclub RT,0,0,0,0,0.024,0,0,0,24.3498,24.3629
2020-02,Ireland,S1,IRLEC,Dialoq - Genesisclub RT,0,0,0,0,0,0,0,0,3.8028,15.1712
2020-02,Italy,S1,ITASI,Dialoq - Genesisclub RT,0,0,0,0,0.04,0.04,0,0,0,0.76387
2020-02,Jordan,S1,JORMC,Dialoq - Genesisclub RT,0,0,0,0,0.64,0,0,0,21.1197,22.6095


Проблемы
1. Не учитываются МО СМС для Стиринга в OCS.
2. Спонсоры предоставляют неправильные тарифы.
3. Проблема с неправильным определением страны пребывания в OCS